In [5]:
import os
import re
from datetime import datetime
import pandas as pd
from newspaper import Article

In [6]:
def get_urls(string):
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+~]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', string) 
    return urls

In [7]:
tabulka = {}
newslettery = os.listdir("nls")
count = 1
i = 0
total = len(newslettery)
for newsletter in newslettery:
    with open("nls" + '/' + newsletter, 'r', encoding="utf8") as n:
        obsah = n.read()
        urls = get_urls(obsah)
        date = os.path.getmtime("nls" + '/' + newsletter)
        date = datetime.fromtimestamp(date).strftime('%Y-%m-%d')
        for u in urls:
            u = u.split(")")[0]
            u = re.split("(^watch)\?", u)[0]
            meta = []
            meta.append(u)
            meta.append(date)
            try:
                article = Article(u)
                article.download()
                article.parse()
                authors = article.authors
                title = article.title
                meta.append(title)
                meta.append(authors)
                if ".com" in u:
                    article.nlp()
                    keywords = article.keywords
                else:
                    keywords = []
                meta.append(keywords)                
            except:
                meta.append(['','',''])
            tabulka[i] = meta
            i += 1
    print(str(count) + "/" + str(total))
    count += 1

FileNotFoundError: [WinError 3] Systém nemůže nalézt uvedenou cestu: 'nls'

In [ ]:
tab = pd.DataFrame.from_dict(tabulka, orient="index", columns=["url","datum","titulek","autorstvo","keywords"]).sort_values(by="datum", ascending=True)

In [ ]:
tab

In [ ]:
with open('clanky.json', 'w', encoding='utf-8') as file:
    tab[["url","titulek","datum","autorstvo","keywords"]].to_json(file, date_format='iso', orient="records", indent=4, force_ascii=False)